# Powerlifting Squat Validator

## 1) Imports and functions to utilize pose-estimator

In [1]:
import cv2
import numpy as np
import os
import tensorflow as tf
import mediapipe as mp

In [2]:
#Initializing Media pipe model and drawing tools
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
# Function to detect key points using mediapipe
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
# Function to draw landmarks of detected keypoints for visualization
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [5]:
#Fucntion to resize input video to fit screen dimensions
def scale_video(scale_percent):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dimensions = (width, height)
    return dimensions

In [6]:
# Function to extract body key-points detected
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose

## 2) Creating Features and Labels for the Model

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [7]:
# Actions we want to predict
Squat_result = np.array(['Valid', 'Invalid'])

In [11]:
# 240 videos
# 120 each for valid and invalid squats
no_of_vids = 120

In [12]:
# Path for exported data, numpy arrays
VALID_SAVE_PATH = os.path.join('Squat_Data')

In [13]:
label_map = {label:num for num, label in enumerate(Squat_result)}

In [14]:
# initializing two arrays to hold features(sequences) and labels(labels)
sequences, labels = [], []
max_frame_num = 0

for action in Squat_result:
    for sequence in range(no_of_vids):
        counter = True
        window = []
        frame_no = 0
        while counter:
            try:
                res = np.load(os.path.join(VALID_SAVE_PATH, action, str(sequence), "{}.npy".format(frame_no)))
                print(sequence, action, frame_no)
                frame_no+=1
                if frame_no>max_frame_num:
                    max_frame_num=frame_no
                window.append(res)
            except:
                break

        sequences.append(window)
        labels.append(label_map[action])

0 Valid 0
0 Valid 1
0 Valid 2
0 Valid 3
0 Valid 4
0 Valid 5
0 Valid 6
0 Valid 7
0 Valid 8
0 Valid 9
0 Valid 10
0 Valid 11
0 Valid 12
0 Valid 13
0 Valid 14
0 Valid 15
0 Valid 16
0 Valid 17
0 Valid 18
0 Valid 19
0 Valid 20
0 Valid 21
0 Valid 22
0 Valid 23
0 Valid 24
0 Valid 25
0 Valid 26
0 Valid 27
0 Valid 28
0 Valid 29
0 Valid 30
0 Valid 31
0 Valid 32
0 Valid 33
0 Valid 34
0 Valid 35
0 Valid 36
0 Valid 37
0 Valid 38
0 Valid 39
0 Valid 40
0 Valid 41
0 Valid 42
0 Valid 43
0 Valid 44
0 Valid 45
0 Valid 46
0 Valid 47
0 Valid 48
0 Valid 49
0 Valid 50
0 Valid 51
0 Valid 52
0 Valid 53
0 Valid 54
0 Valid 55
0 Valid 56
0 Valid 57
0 Valid 58
0 Valid 59
0 Valid 60
0 Valid 61
0 Valid 62
0 Valid 63
0 Valid 64
0 Valid 65
0 Valid 66
0 Valid 67
0 Valid 68
0 Valid 69
0 Valid 70
0 Valid 71
0 Valid 72
0 Valid 73
0 Valid 74
0 Valid 75
0 Valid 76
0 Valid 77
0 Valid 78
0 Valid 79
0 Valid 80
0 Valid 81
0 Valid 82
0 Valid 83
0 Valid 84
0 Valid 85
0 Valid 86
0 Valid 87
0 Valid 88
0 Valid 89
0 Valid 90
0 Valid 9

7 Valid 51
7 Valid 52
7 Valid 53
7 Valid 54
7 Valid 55
7 Valid 56
7 Valid 57
7 Valid 58
7 Valid 59
7 Valid 60
7 Valid 61
7 Valid 62
7 Valid 63
7 Valid 64
7 Valid 65
7 Valid 66
7 Valid 67
7 Valid 68
7 Valid 69
7 Valid 70
7 Valid 71
7 Valid 72
7 Valid 73
7 Valid 74
7 Valid 75
7 Valid 76
7 Valid 77
7 Valid 78
7 Valid 79
7 Valid 80
7 Valid 81
7 Valid 82
7 Valid 83
7 Valid 84
7 Valid 85
7 Valid 86
7 Valid 87
7 Valid 88
7 Valid 89
7 Valid 90
7 Valid 91
7 Valid 92
7 Valid 93
7 Valid 94
7 Valid 95
7 Valid 96
7 Valid 97
7 Valid 98
7 Valid 99
7 Valid 100
7 Valid 101
7 Valid 102
7 Valid 103
7 Valid 104
7 Valid 105
8 Valid 0
8 Valid 1
8 Valid 2
8 Valid 3
8 Valid 4
8 Valid 5
8 Valid 6
8 Valid 7
8 Valid 8
8 Valid 9
8 Valid 10
8 Valid 11
8 Valid 12
8 Valid 13
8 Valid 14
8 Valid 15
8 Valid 16
8 Valid 17
8 Valid 18
8 Valid 19
8 Valid 20
8 Valid 21
8 Valid 22
8 Valid 23
8 Valid 24
8 Valid 25
8 Valid 26
8 Valid 27
8 Valid 28
8 Valid 29
8 Valid 30
8 Valid 31
8 Valid 32
8 Valid 33
8 Valid 34
8 Valid 35
8 V

15 Valid 69
15 Valid 70
15 Valid 71
15 Valid 72
15 Valid 73
15 Valid 74
15 Valid 75
15 Valid 76
15 Valid 77
15 Valid 78
15 Valid 79
15 Valid 80
15 Valid 81
15 Valid 82
15 Valid 83
15 Valid 84
16 Valid 0
16 Valid 1
16 Valid 2
16 Valid 3
16 Valid 4
16 Valid 5
16 Valid 6
16 Valid 7
16 Valid 8
16 Valid 9
16 Valid 10
16 Valid 11
16 Valid 12
16 Valid 13
16 Valid 14
16 Valid 15
16 Valid 16
16 Valid 17
16 Valid 18
16 Valid 19
16 Valid 20
16 Valid 21
16 Valid 22
16 Valid 23
16 Valid 24
16 Valid 25
16 Valid 26
16 Valid 27
16 Valid 28
16 Valid 29
16 Valid 30
16 Valid 31
16 Valid 32
16 Valid 33
16 Valid 34
16 Valid 35
16 Valid 36
16 Valid 37
16 Valid 38
16 Valid 39
16 Valid 40
16 Valid 41
16 Valid 42
16 Valid 43
16 Valid 44
16 Valid 45
16 Valid 46
16 Valid 47
16 Valid 48
16 Valid 49
16 Valid 50
16 Valid 51
16 Valid 52
16 Valid 53
16 Valid 54
16 Valid 55
16 Valid 56
16 Valid 57
16 Valid 58
16 Valid 59
16 Valid 60
16 Valid 61
16 Valid 62
16 Valid 63
16 Valid 64
16 Valid 65
16 Valid 66
16 Valid 67
16

23 Valid 62
23 Valid 63
23 Valid 64
23 Valid 65
23 Valid 66
23 Valid 67
23 Valid 68
23 Valid 69
23 Valid 70
23 Valid 71
23 Valid 72
23 Valid 73
23 Valid 74
23 Valid 75
23 Valid 76
23 Valid 77
23 Valid 78
23 Valid 79
23 Valid 80
23 Valid 81
23 Valid 82
24 Valid 0
24 Valid 1
24 Valid 2
24 Valid 3
24 Valid 4
24 Valid 5
24 Valid 6
24 Valid 7
24 Valid 8
24 Valid 9
24 Valid 10
24 Valid 11
24 Valid 12
24 Valid 13
24 Valid 14
24 Valid 15
24 Valid 16
24 Valid 17
24 Valid 18
24 Valid 19
24 Valid 20
24 Valid 21
24 Valid 22
24 Valid 23
24 Valid 24
24 Valid 25
24 Valid 26
24 Valid 27
24 Valid 28
24 Valid 29
24 Valid 30
24 Valid 31
24 Valid 32
24 Valid 33
24 Valid 34
24 Valid 35
24 Valid 36
24 Valid 37
24 Valid 38
24 Valid 39
24 Valid 40
24 Valid 41
24 Valid 42
24 Valid 43
24 Valid 44
24 Valid 45
24 Valid 46
24 Valid 47
24 Valid 48
24 Valid 49
24 Valid 50
24 Valid 51
24 Valid 52
24 Valid 53
24 Valid 54
24 Valid 55
24 Valid 56
24 Valid 57
24 Valid 58
24 Valid 59
24 Valid 60
24 Valid 61
24 Valid 62
24

30 Valid 139
30 Valid 140
30 Valid 141
30 Valid 142
30 Valid 143
30 Valid 144
30 Valid 145
30 Valid 146
30 Valid 147
30 Valid 148
30 Valid 149
30 Valid 150
30 Valid 151
30 Valid 152
30 Valid 153
30 Valid 154
30 Valid 155
30 Valid 156
31 Valid 0
31 Valid 1
31 Valid 2
31 Valid 3
31 Valid 4
31 Valid 5
31 Valid 6
31 Valid 7
31 Valid 8
31 Valid 9
31 Valid 10
31 Valid 11
31 Valid 12
31 Valid 13
31 Valid 14
31 Valid 15
31 Valid 16
31 Valid 17
31 Valid 18
31 Valid 19
31 Valid 20
31 Valid 21
31 Valid 22
31 Valid 23
31 Valid 24
31 Valid 25
31 Valid 26
31 Valid 27
31 Valid 28
31 Valid 29
31 Valid 30
31 Valid 31
31 Valid 32
31 Valid 33
31 Valid 34
31 Valid 35
31 Valid 36
31 Valid 37
31 Valid 38
31 Valid 39
31 Valid 40
31 Valid 41
31 Valid 42
31 Valid 43
31 Valid 44
31 Valid 45
31 Valid 46
31 Valid 47
31 Valid 48
31 Valid 49
31 Valid 50
31 Valid 51
31 Valid 52
31 Valid 53
31 Valid 54
31 Valid 55
31 Valid 56
31 Valid 57
31 Valid 58
31 Valid 59
31 Valid 60
31 Valid 61
31 Valid 62
31 Valid 63
31 Valid

...

86 Invalid 0
86 Invalid 1
86 Invalid 2
86 Invalid 3
86 Invalid 4
86 Invalid 5
86 Invalid 6
86 Invalid 7
86 Invalid 8
86 Invalid 9
86 Invalid 10
86 Invalid 11
86 Invalid 12
86 Invalid 13
86 Invalid 14
86 Invalid 15
86 Invalid 16
86 Invalid 17
86 Invalid 18
86 Invalid 19
86 Invalid 20
86 Invalid 21
86 Invalid 22
86 Invalid 23
86 Invalid 24
86 Invalid 25
86 Invalid 26
86 Invalid 27
86 Invalid 28
86 Invalid 29
86 Invalid 30
86 Invalid 31
86 Invalid 32
86 Invalid 33
86 Invalid 34
86 Invalid 35
86 Invalid 36
86 Invalid 37
86 Invalid 38
86 Invalid 39
86 Invalid 40
86 Invalid 41
86 Invalid 42
86 Invalid 43
86 Invalid 44
86 Invalid 45
86 Invalid 46
86 Invalid 47
86 Invalid 48
86 Invalid 49
86 Invalid 50
86 Invalid 51
86 Invalid 52
86 Invalid 53
87 Invalid 0
87 Invalid 1
87 Invalid 2
87 Invalid 3
87 Invalid 4
87 Invalid 5
87 Invalid 6
87 Invalid 7
87 Invalid 8
87 Invalid 9
87 Invalid 10
87 Invalid 11
87 Invalid 12
87 Invalid 13
87 Invalid 14
87 Invalid 15
87 Invalid 16
87 Invalid 17
87 Invalid 1

95 Invalid 6
95 Invalid 7
95 Invalid 8
95 Invalid 9
95 Invalid 10
95 Invalid 11
95 Invalid 12
95 Invalid 13
95 Invalid 14
95 Invalid 15
95 Invalid 16
95 Invalid 17
95 Invalid 18
95 Invalid 19
95 Invalid 20
95 Invalid 21
95 Invalid 22
95 Invalid 23
95 Invalid 24
95 Invalid 25
95 Invalid 26
95 Invalid 27
95 Invalid 28
95 Invalid 29
95 Invalid 30
95 Invalid 31
95 Invalid 32
95 Invalid 33
95 Invalid 34
95 Invalid 35
95 Invalid 36
95 Invalid 37
95 Invalid 38
95 Invalid 39
95 Invalid 40
95 Invalid 41
95 Invalid 42
95 Invalid 43
95 Invalid 44
95 Invalid 45
95 Invalid 46
95 Invalid 47
95 Invalid 48
95 Invalid 49
95 Invalid 50
95 Invalid 51
95 Invalid 52
95 Invalid 53
95 Invalid 54
95 Invalid 55
95 Invalid 56
95 Invalid 57
95 Invalid 58
95 Invalid 59
95 Invalid 60
95 Invalid 61
95 Invalid 62
95 Invalid 63
95 Invalid 64
95 Invalid 65
95 Invalid 66
95 Invalid 67
95 Invalid 68
95 Invalid 69
95 Invalid 70
95 Invalid 71
95 Invalid 72
95 Invalid 73
95 Invalid 74
95 Invalid 75
95 Invalid 76
95 Invalid

102 Invalid 81
102 Invalid 82
102 Invalid 83
102 Invalid 84
102 Invalid 85
102 Invalid 86
102 Invalid 87
102 Invalid 88
102 Invalid 89
102 Invalid 90
102 Invalid 91
103 Invalid 0
103 Invalid 1
103 Invalid 2
103 Invalid 3
103 Invalid 4
103 Invalid 5
103 Invalid 6
103 Invalid 7
103 Invalid 8
103 Invalid 9
103 Invalid 10
103 Invalid 11
103 Invalid 12
103 Invalid 13
103 Invalid 14
103 Invalid 15
103 Invalid 16
103 Invalid 17
103 Invalid 18
103 Invalid 19
103 Invalid 20
103 Invalid 21
103 Invalid 22
103 Invalid 23
103 Invalid 24
103 Invalid 25
103 Invalid 26
103 Invalid 27
103 Invalid 28
103 Invalid 29
103 Invalid 30
103 Invalid 31
103 Invalid 32
103 Invalid 33
103 Invalid 34
103 Invalid 35
103 Invalid 36
103 Invalid 37
103 Invalid 38
103 Invalid 39
103 Invalid 40
103 Invalid 41
103 Invalid 42
103 Invalid 43
103 Invalid 44
103 Invalid 45
103 Invalid 46
103 Invalid 47
103 Invalid 48
103 Invalid 49
103 Invalid 50
103 Invalid 51
103 Invalid 52
103 Invalid 53
103 Invalid 54
103 Invalid 55
103 I

110 Invalid 4
110 Invalid 5
110 Invalid 6
110 Invalid 7
110 Invalid 8
110 Invalid 9
110 Invalid 10
110 Invalid 11
110 Invalid 12
110 Invalid 13
110 Invalid 14
110 Invalid 15
110 Invalid 16
110 Invalid 17
110 Invalid 18
110 Invalid 19
110 Invalid 20
110 Invalid 21
110 Invalid 22
110 Invalid 23
110 Invalid 24
110 Invalid 25
110 Invalid 26
110 Invalid 27
110 Invalid 28
110 Invalid 29
110 Invalid 30
110 Invalid 31
110 Invalid 32
110 Invalid 33
110 Invalid 34
110 Invalid 35
110 Invalid 36
110 Invalid 37
110 Invalid 38
110 Invalid 39
110 Invalid 40
110 Invalid 41
110 Invalid 42
110 Invalid 43
110 Invalid 44
110 Invalid 45
110 Invalid 46
110 Invalid 47
110 Invalid 48
110 Invalid 49
110 Invalid 50
110 Invalid 51
110 Invalid 52
110 Invalid 53
110 Invalid 54
110 Invalid 55
110 Invalid 56
110 Invalid 57
110 Invalid 58
110 Invalid 59
110 Invalid 60
110 Invalid 61
110 Invalid 62
110 Invalid 63
110 Invalid 64
110 Invalid 65
110 Invalid 66
110 Invalid 67
110 Invalid 68
110 Invalid 69
110 Invalid 70
1

117 Invalid 19
117 Invalid 20
117 Invalid 21
117 Invalid 22
117 Invalid 23
117 Invalid 24
117 Invalid 25
117 Invalid 26
117 Invalid 27
117 Invalid 28
117 Invalid 29
117 Invalid 30
117 Invalid 31
117 Invalid 32
117 Invalid 33
117 Invalid 34
117 Invalid 35
117 Invalid 36
117 Invalid 37
117 Invalid 38
117 Invalid 39
117 Invalid 40
117 Invalid 41
117 Invalid 42
117 Invalid 43
117 Invalid 44
117 Invalid 45
117 Invalid 46
117 Invalid 47
117 Invalid 48
117 Invalid 49
117 Invalid 50
117 Invalid 51
117 Invalid 52
117 Invalid 53
117 Invalid 54
117 Invalid 55
117 Invalid 56
117 Invalid 57
117 Invalid 58
117 Invalid 59
117 Invalid 60
117 Invalid 61
117 Invalid 62
117 Invalid 63
117 Invalid 64
117 Invalid 65
117 Invalid 66
117 Invalid 67
117 Invalid 68
117 Invalid 69
117 Invalid 70
117 Invalid 71
117 Invalid 72
117 Invalid 73
117 Invalid 74
117 Invalid 75
117 Invalid 76
117 Invalid 77
117 Invalid 78
117 Invalid 79
117 Invalid 80
118 Invalid 0
118 Invalid 1
118 Invalid 2
118 Invalid 3
118 Invalid 4


In [15]:
# Each feature in sequences holds a shape of (X,132)
# X being the number of frames of the video
# As we are using an LSTM model it requires all input to be the same size
# We are effectively doing that by utilizing the keras pad_sequences function
# Which pads all the features to the same size as the longest video
# Which is a 191 frames in our scenario

X = tf.keras.preprocessing.sequence.pad_sequences(sequences)

In [16]:
# We now curretly have 240 features(videos)
# Each of the shape of 191(frames) and 132(key-points)

X.shape

(240, 191, 132)

In [17]:
y = to_categorical(labels).astype(int)
y.shape

(240, 2)

In [ ]:
# Saving Features and Labels for later use
np.save("Features",X)
np.save("Labels",y)

## 3) Creating, Training and Testing the Model

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [27]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [23]:
# Setting callbacks for tensorboard
log_dir = os.path.join('Logs_Retrain')
tb_callback = TensorBoard(log_dir=log_dir)

In [20]:
# Basic LSTM model for demonstartion purposes

model = Sequential()
model.add(LSTM(64, return_sequences=False, activation='relu', input_shape=(191,132)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(Squat_result.shape[0], activation='sigmoid'))

In [21]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                50432     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 56,738
Trainable params: 56,738
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[tb_callback])

Epoch 1/100
6/6 [==============================] - 1s 90ms/step - loss: 0.6940 - accuracy: 0.5312 - val_loss: 0.6901 - val_accuracy: 0.5625
Epoch 2/100
6/6 [==============================] - 0s 52ms/step - loss: 0.6873 - accuracy: 0.5781 - val_loss: 0.6801 - val_accuracy: 0.5625
Epoch 3/100
6/6 [==============================] - 0s 51ms/step - loss: 32476210.0000 - accuracy: 0.5573 - val_loss: 38374.4258 - val_accuracy: 0.4792
Epoch 4/100
6/6 [==============================] - 0s 48ms/step - loss: 7104468.5000 - accuracy: 0.5104 - val_loss: 125168.8047 - val_accuracy: 0.4792
Epoch 5/100
6/6 [==============================] - 0s 46ms/step - loss: 5233701.5000 - accuracy: 0.5104 - val_loss: 14417.1104 - val_accuracy: 0.5000
Epoch 6/100
6/6 [==============================] - 0s 49ms/step - loss: 492124.5000 - accuracy: 0.5208 - val_loss: 3538.8547 - val_accuracy: 0.6250
Epoch 7/100
6/6 [==============================] - 0s 50ms/step - loss: 238595.8750 - accuracy: 0.6562 - val_loss: 18899

6/6 [==============================] - 0s 46ms/step - loss: 0.6320 - accuracy: 0.5781 - val_loss: 415196.2188 - val_accuracy: 0.5208
Epoch 59/100
6/6 [==============================] - 0s 46ms/step - loss: 0.6280 - accuracy: 0.5781 - val_loss: 1671272.8750 - val_accuracy: 0.5208
Epoch 60/100
6/6 [==============================] - 0s 47ms/step - loss: 0.6236 - accuracy: 0.5781 - val_loss: 4917334.5000 - val_accuracy: 0.5208
Epoch 61/100
6/6 [==============================] - 0s 47ms/step - loss: 0.6208 - accuracy: 0.5781 - val_loss: 11366180.0000 - val_accuracy: 0.5208
Epoch 62/100
6/6 [==============================] - 0s 47ms/step - loss: 0.6184 - accuracy: 0.5781 - val_loss: 27590798.0000 - val_accuracy: 0.5208
Epoch 63/100
6/6 [==============================] - 0s 47ms/step - loss: 0.6150 - accuracy: 0.5781 - val_loss: 192628288.0000 - val_accuracy: 0.5208
Epoch 64/100
6/6 [==============================] - 0s 46ms/step - loss: 0.6107 - accuracy: 0.5781 - val_loss: 590567744.0000 - 

### 3.1) Saving model weights

In [25]:
model.save('model.h5')

In [28]:
saved_model = load_model('model.h5')

## 4) Evaluation of Model

In [29]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [30]:
yhat = saved_model.predict(X_test)

In [31]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [32]:
confusion_matrix(ytrue, yhat)

array([[17,  6],
       [21,  4]], dtype=int64)

In [33]:
accuracy_score(ytrue, yhat)

0.4375

## 5) Making predictions to new videos

In [34]:
# Array is used to pad the recieved sequence to the shape the Deep Learning Model Accepts
longest_sequence = np.load("Longest_Sequence.npy")

In [40]:
# New detection variables
sequence_for_prediction = [longest_sequence]

window = []

# Capturing Video Stream
cap = cv2.VideoCapture(r'your video path here')


# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        #Close window at end of video
        if not ret:
            break

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        dim = scale_video(30)
        # Show to screen

        image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)


        cv2.imshow('OpenCV Feed', image)

        # Prediction logic
        keypoints = extract_keypoints(results)
        window.append(keypoints)
        
        

        

    cap.release()
    cv2.destroyAllWindows()

    sequence_for_prediction.append(window)
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence_for_prediction, maxlen=191)
    seq_to_predict = padded_sequence[1]       


In [39]:
print (seq_to_predict)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [41]:
# Predicting the squat validity
res = model.predict(np.expand_dims(seq_to_predict, axis=0))
print (Squat_result[np.argmax(res)])

Valid
